## clone repo & download dataset

In [ ]:
!git clone https://github.com/Yui-Arthur/TFLM_with_keyword_spotting.git
%cd TFLM_with_keyword_spotting

In [ ]:
!mkdir speech_commands
!wget http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
!tar -C speech_commands -zxvf speech_commands_v0.02.tar.gz > /dev/null

In [1]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import logging
import sys   
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from train_tf import logger_setting , google_speech_commands_dataset , show_train_results , testing_model , train_one_epoch , conv_model


## training

In [2]:
input_dim = (16000,1)
wav_size = 16000
output_class = 35
# hyperparameter
epochs = 10
batch_size = 64
learning_rate = 1e-3

# path & logger setting
speech_commands_root_folder = Path("./speech_commands")
root_folder = Path("train")
logger : logging.Logger = logger_setting(root_folder , "train")

# model init
model = conv_model(input_dim , output_class , learning_rate)
model.build((batch_size,16000,1))
model._model.summary()
# exit()
# logger.info(f"Model Parameters : {num_params}")

# get dataloader
train_dataloader , valid_dataloader , test_dataloader = google_speech_commands_dataset(speech_commands_root_folder, wav_size, batch_size)
# train_dataloader , valid_dataloader , test_dataloader = google_speech_commands_dataset(speech_commands_root_folder, wav_size, batch_size , (64,64,64))

# train / valid
train_info = []
max_acc = -1
for epoch in range(epochs):
    logger.info(f"epoch {epoch} :")
    epoch_info = train_one_epoch(model , train_dataloader , valid_dataloader , max_acc , logger , root_folder)
    train_info.append(list(epoch_info))
    max_acc = max(epoch_info[2] ,max_acc)

# show train the result    
show_train_results(train_info , root_folder)

# test the model
testing_model(test_dataloader , logger , root_dir=root_folder)




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16000, 1)]        0         
                                                                 
 conv1d (Conv1D)             (None, 4000, 32)          1056      
                                                                 
 re_lu (ReLU)                (None, 4000, 32)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 1000, 64)          65600     
                                                                 
 re_lu_1 (ReLU)              (None, 1000, 64)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 250, 128)          262272    
                                                                 
 re_lu_2 (ReLU)              (None, 250, 128)          0  

  1%|          | 9/1325 [00:06<16:39,  1.32it/s]


KeyboardInterrupt: 